In [6]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

In [3]:
DATA = '../../../data'

ibm = pd.read_csv(f'{DATA}/IBM.csv')
ibm = ibm[['Date', 'Adj Close']]
ibm.rename(columns={'Adj Close': 'Close'}, inplace=True)
ibm.head()

,Date,Close
0,2001-01-02,56.790684
1,2001-01-03,63.361145
2,2001-01-04,62.398586
3,2001-01-05,62.942654
4,2001-01-08,62.649712


In [4]:
ibm.shape

(4718, 2)

In [5]:
mean1, mean2 = ibm.iloc[:2359].Close.mean(), ibm.iloc[2359:].Close.mean()
var1, var2 = ibm.iloc[:2359].Close.var(), ibm.iloc[2359:].Close.var()
print(f'Means: {mean1:.2f}, {mean2:.2f}')
print(f'Vars: {var1:.2f}, {var2:.2f}')

Means: 67.81, 137.18
Vars: 168.09, 248.29


In [9]:
adf_result = adfuller(ibm.Close.tolist())
print(f'ADF: {adf_result[0]:.2f}, (p={adf_result[1]:.2f})')

ADF: -1.38, (p=0.59)


More negative ADFs more likely to be stationary
p = 0.59: cannot reject null: (i.e., most likely is non-stationary)